In [59]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('/Users/MYK/Desktop/IRES/scripts')

import matplotlib.pyplot as plt
import numpy as np
import pywt
import scipy.stats
from collections import Counter
import utils as u

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Need larger dataset!

In [60]:
# Functions copied from http://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [61]:
def get_dwt_features(profs, wavelet='haar', level=None, nfeatures=None):
    '''Function to decompose each profile, get features at each level and return 
    a feature matrix which will be nprofs X nfeatures.
    
    Parameters
    ----------
    
    profs : 2d array
    NumPy 2d array, each row corresponds to profile data.
    nprofs X nbins
    
    wavelet : str
    Wavelet name to use in wavelet decomposition. Must come from
    pywt.wavelist().
    
    max_level : int
    Maximum level to apply wavelet decomposition to. If None,
    pywt will automatically use maximum level possible.
    
    nfeatures : int
    Number of features. Set to 12 by default if using
    functions from Ahmet Taspinar's website.
    
    Returns
    -------
    matrix : 2d array
    Matrix containing features per profile. nprofs X nfeatures.
    '''
    
    nprofs = len(profs)
    
    #feature_matrix
    if nfeatures is None:
        if level is None:
            max_level =  pywt.dwt_max_level(len(profs[0]), wavelet)
            nfeatures = len(get_features(profs[0])) * (max_level+1)
        else:
             nfeatures = len(get_features(profs[0])) * (level+1)
    matrix = np.zeros((nprofs, nfeatures))
    
    for i in range(len(profs)):
        features = []
        coeffs = pywt.wavedec(profs[i], wavelet, mode='periodization', level=level)
        for c in coeffs:
            features += get_features(c)
        matrix[i] = features
    
    return matrix

In [62]:
profs = np.loadtxt('data/J0332_profs.txt')

In [63]:
abnormal_ind = np.array([9, 16, 17, 18, 22, 31, 32, 35, 38, 40, 52, 56])

y = [1 if ind in abnormal_ind else 0 for ind in range(len(profs))]
X = get_dwt_features(profs)

In [64]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

### Random Foreset Classifier

In [90]:
rnd_clf = RandomForestClassifier(max_leaf_nodes=10)
rnd_clf.fit(X_train, y_train)

cross_val_score(rnd_clf, X_train, y_train, cv=10)

/Users/MYK/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/MYK/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.8       , 1.        , 0.8       , 1.        , 0.8       ,
       0.8       , 1.        , 1.        , 0.75      , 0.66666667])

In [78]:
y_pred = rnd_clf.predict(X_test)

In [79]:
accuracy_score(y_test, y_pred)

1.0

In [80]:
for i,j in zip(y_test, y_pred):
    print(i,j)

0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
1 1
0 0


### SVM Classifier
Needs features scaling

In [81]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [82]:
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(C=10, kernel='linear'))
])

svm_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [83]:
cross_val_score(svm_pipeline, X_train, y_train, cv=10, scoring='accuracy')

/Users/MYK/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.8       , 1.        , 1.        , 1.        , 1.        ,
       0.8       , 1.        , 1.        , 0.5       , 0.66666667])

In [84]:
y_pred = svm_pipeline.predict(X_test)

In [85]:
svm_pipeline.score(X_test, y_test)

1.0

In [86]:
for i,j in zip(y_test, y_pred):
    print(i, j)

0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
1 1
0 0


In [87]:
isort = np.argsort(svm_pipeline[1].coef_)[0]

In [88]:
features = ['entropy', 'zero_cross', 'mean_cross', 'n5', 'n25', 'n75', 'n95', 'median', 'mean', 'std', 'var', 'rms']

In [89]:
#for each level of coeff
for i in range(len(isort)):
    if isort[i] >= 100:
        print('Level: {}, feature: {}'.format(i//12 + 1, features[i%12]))

Level: 1, feature: rms
Level: 2, feature: mean
Level: 3, feature: mean_cross
Level: 3, feature: n5
Level: 4, feature: entropy
Level: 4, feature: zero_cross
Level: 4, feature: n95
Level: 4, feature: rms
Level: 6, feature: median
Level: 6, feature: std
Level: 7, feature: zero_cross
Level: 7, feature: mean_cross
Level: 7, feature: var
Level: 8, feature: n5
Level: 8, feature: n25
Level: 8, feature: mean
Level: 9, feature: entropy
Level: 9, feature: n25
Level: 9, feature: std
Level: 9, feature: rms
